In [ ]:
# Install necessary packages
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install fasttext
!pip install scikit-learn
!pip install nltk
!pip install rouge-score
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=68103cd46aa4b39912a4e994a63f02b019d0978d0fd46c7fd9c4e5d41059675b
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
# Download English FastText vectors
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gunzip cc.en.300.vec.gz

# Download Persian FastText vectors
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.vec.gz
!gunzip cc.fa.300.vec.gz

--2024-09-11 09:02:18--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.238.176.126, 18.238.176.44, 18.238.176.19, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.238.176.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G  63.4MB/s    in 17s     

2024-09-11 09:02:35 (75.4 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]

--2024-09-11 09:03:35--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.238.176.19, 18.238.176.44, 18.238.176.115, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.238.176.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1258183862 (1.2G) [binary/octet-stream]
Saving to

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, Input, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler
from sklearn.decomposition import PCA
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import psutil

# Check memory usage
print(f"Memory Usage: {psutil.virtual_memory().percent}%")

# Function to load FastText vectors from .vec file
def load_fasttext_vectors(filepath):
    embeddings_index = {}
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            values = line.rstrip().split(' ')
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Load FastText vectors
english_fasttext = load_fasttext_vectors('cc.en.300.vec')
persian_fasttext = load_fasttext_vectors('cc.fa.300.vec')

# Load the preprocessed data
data = pd.read_csv('df_open_10000.csv')
print(data.head())

# Hyperparameters
MAX_VOCAB_SIZE = 5000
MAX_SEQUENCE_LENGTH = 50
embedding_dim = 300
d_model = 300
num_layers = 2
num_heads = 4
dropout_rate = 0.2
batch_size = 32
accumulation_steps = 4
epochs=15

# Tokenizer for English
english_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
english_tokenizer.fit_on_texts(data['English'])
english_sequences = english_tokenizer.texts_to_sequences(data['English'])

# Tokenizer for Persian
persian_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
persian_tokenizer.fit_on_texts(data['Persian'])
persian_sequences = persian_tokenizer.texts_to_sequences(data['Persian'])

# Padding sequences
english_padded = pad_sequences(english_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
persian_padded = pad_sequences(persian_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp, idx_train, idx_temp = train_test_split(
    english_padded, persian_padded, data.index, test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test, idx_val, idx_test = train_test_split(
    X_temp, y_temp, idx_temp, test_size=0.5, random_state=42)

# Create the embedding matrix for English
def create_embedding_matrix(tokenizer, embedding_dim, embeddings_index):
    embedding_matrix = np.zeros((MAX_VOCAB_SIZE, embedding_dim))
    for word, i in tokenizer.word_index.items():
        if i < MAX_VOCAB_SIZE:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
            else:
                embedding_matrix[i] = np.zeros(embedding_dim)
    return embedding_matrix

# Create embedding matrices for both English and Persian
english_embedding_matrix = create_embedding_matrix(english_tokenizer, embedding_dim, english_fasttext)
persian_embedding_matrix = create_embedding_matrix(persian_tokenizer, embedding_dim, persian_fasttext)

Memory Usage: 87.5%
                                             English  \
0                         the passion of joan of arc   
1                           a film by carl th dreyer   
2  at the bibliotheque de la chambre des deputes ...   
3  the record of the trial of joan of arc the tri...   
4  the questions of the judges and joans response...   

                                             Persian         Source  
0                                      مصایب ژاندارک  OpenSubtitles  
1                         فیلمی از کارل تیودور درایر  OpenSubtitles  
2  در کتابخانه مجلس نمایندگان پاریس یکی از خارق ا...  OpenSubtitles  
3  پرونده محاکمه ژاندارک محاکمه ای که منجر به مرگ...  OpenSubtitles  
4        پرسشهای قضات و پاسخهای ژان عینا پرونده شدند  OpenSubtitles  


In [ ]:
# Transformer model with improvements
def get_positional_encoding(seq_len, d_model):
    angles = np.arange(seq_len)[:, np.newaxis] / np.power(10000, 2 * (np.arange(d_model)[np.newaxis, :] // 2) / d_model)
    pos_encoding = np.zeros((seq_len, d_model))
    pos_encoding[:, 0::2] = np.sin(angles[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(angles[:, 1::2])
    return tf.cast(pos_encoding, dtype=tf.float32)

class TransformerModel(tf.keras.Model):
    def __init__(self, vocab_size, num_layers, d_model, num_heads, dropout, embedding_matrix, name="transformer"):
        super(TransformerModel, self).__init__(name=name)
        self.embedding = Embedding(vocab_size, d_model, weights=[embedding_matrix], trainable=True)
        self.pos_encoding = get_positional_encoding(MAX_SEQUENCE_LENGTH, d_model)
        self.enc_layers = [
            {
                "attention": tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model),
                "norm": LayerNormalization(epsilon=1e-6),
                "ffn": Dense(d_model, activation='relu')
            }
            for _ in range(num_layers)
        ]
        self.dropout = Dropout(dropout)

    def call(self, inputs):
        x = self.embedding(inputs)
        x += self.pos_encoding[:tf.shape(x)[1], :]
        x = self.dropout(x)
        for layer in self.enc_layers:
            attn_output = layer["attention"](x, x)
            attn_output = layer["norm"](attn_output + x)
            x = layer["ffn"](attn_output)
        return x

def build_transformer(vocab_size, d_model, num_layers, num_heads, dropout_rate, embedding_matrix):
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH,))
    transformer_model = TransformerModel(vocab_size, num_layers, d_model, num_heads, dropout_rate, embedding_matrix)(inputs)
    outputs = Dense(vocab_size, activation='softmax')(transformer_model)
    model = Model(inputs, outputs)
    return model

# Build the transformer model for English
vocab_size = MAX_VOCAB_SIZE
model = build_transformer(vocab_size, d_model, num_layers, num_heads, dropout_rate, english_embedding_matrix)

# Learning rate scheduler with warm-up
def lr_scheduler(epoch, lr):
    warmup_epochs = 5
    if epoch < warmup_epochs:
        return float(lr * (epoch + 1) / warmup_epochs)  # Ensure the learning rate is a float
    else:
        return float(lr * tf.math.exp(-0.1))  # Ensure the learning rate is a float


# Compile the model without label smoothing
optimizer = Adam(learning_rate=0.0005)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Prepare target data (Shift Persian sequences by one step)
decoder_target_data = np.zeros_like(y_train)
decoder_target_data[:, :-1] = y_train[:, 1:]
decoder_target_data[:, -1] = 0

# Define callbacks
checkpoint_filepath = 'epoch-{epoch:02d}-val_loss-{val_loss:.2f}.keras'
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode="min")
lr_schedule = LearningRateScheduler(lr_scheduler)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# Train the model
history = model.fit(
    X_train,
    np.expand_dims(decoder_target_data, -1),
    epochs=epochs,
    batch_size=batch_size // accumulation_steps,
    validation_data=(X_val, np.expand_dims(y_val, -1)),
    callbacks=[early_stopping, lr_schedule, checkpoint]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, np.expand_dims(y_test, -1))
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:609: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 564ms/step - accuracy: 0.8648 - loss: 1.6328
Epoch 1: val_loss improved from inf to 1.03581, saving model to epoch-01-val_loss-1.04.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 600s 592ms/step - accuracy: 0.8649 - loss: 1.6322 - val_accuracy: 0.8657 - val_loss: 1.0358 - learning_rate: 1.0000e-04
Epoch 2/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 552ms/step - accuracy: 0.8873 - loss: 0.8646
Epoch 2: val_loss improved from 1.03581 to 1.03540, saving model to epoch-02-val_loss-1.04.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 598s 598ms/step - accuracy: 0.8873 - loss: 0.8646 - val_accuracy: 0.8662 - val_loss: 1.0354 - learning_rate: 4.0000e-05
Epoch 3/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 557ms/step - accuracy: 0.8850 - loss: 0.8716
Epoch 3: val_loss improved from 1.03540 to 1.02924, saving model to epoch-03-val_loss-1.03.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 607s 584ms/step - accuracy: 0.8850 - loss: 0.8716 - val_accuracy: 0.8663 - val_loss: 1.0292 - learning_rate: 2.4000e-05
E

In [ ]:
beam_width = 3

# Beam search decoding
def beam_search_decoding(model, input_seq, beam_width=3):
    sequences = [[list(), 0.0]]  # Start with an empty sequence and a probability of 0
    for row in input_seq:
        all_candidates = []
        for seq, score in sequences:
            # Predict next words
            yhat = model.predict(np.expand_dims(np.expand_dims(row, axis=0), axis=-1), verbose=0)
            yhat = yhat[0, -1, :]  # Extract the last time step prediction probabilities

            # Find top beam_width candidates
            top_indices = np.argsort(yhat)[-beam_width:]  # Select top k candidates

            for j in top_indices:
                candidate = [seq + [j], score + np.log(yhat[j])]  # Append predicted word and its log probability
                all_candidates.append(candidate)

        # Order all candidates by their scores and select the best beam_width sequences
        ordered = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)
        sequences = ordered[:beam_width]

    return sequences[0][0]  # Return the best sequence

# Limit the number of test samples to avoid memory issues
num_test_samples = 50  # Adjust this number based on available memory
X_test_subset = X_test[:num_test_samples]
y_test_subset = y_test[:num_test_samples]

# Predictions and decoding
y_pred = [beam_search_decoding(model, seq, beam_width=beam_width) for seq in X_test_subset]

# Decoding sequences
def decode_sequence(sequence, tokenizer):
    reverse_word_map = {index: word for word, index in tokenizer.word_index.items()}
    decoded_sentence = ' '.join([reverse_word_map.get(idx, '') for idx in sequence if idx != 0])
    return decoded_sentence

y_pred_decoded = [decode_sequence(seq, persian_tokenizer) for seq in y_pred]
y_true_decoded = [decode_sequence(seq, persian_tokenizer) for seq in y_test_subset]

# Evaluate BLEU score
def evaluate_bleu_score(y_true, y_pred):
    references = [[sentence] for sentence in y_true]
    hypotheses = [sentence for sentence in y_pred]
    return corpus_bleu(references, hypotheses)

bleu_score = evaluate_bleu_score(y_true_decoded, y_pred_decoded)
print(f"BLEU Score: {bleu_score}")

# Evaluate ROUGE score
def evaluate_rouge_score(y_true, y_pred):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(true, pred) for true, pred in zip(y_true, y_pred)]
    avg_rouge = {
        "rouge1": np.mean([score['rouge1'].fmeasure for score in rouge_scores]),
        "rougeL": np.mean([score['rougeL'].fmeasure for score in rouge_scores]),
    }
    return avg_rouge

rouge_score = evaluate_rouge_score(y_true_decoded, y_pred_decoded)
print(f"ROUGE-1: {rouge_score['rouge1']}, ROUGE-L: {rouge_score['rougeL']}")

# Save predictions
output_data = []
for i in range(len(X_test_subset)):
    original_english = data['English'].iloc[idx_test[i]]
    predicted_persian = decode_sequence(y_pred[i], persian_tokenizer)
    actual_persian = decode_sequence(y_test_subset[i], persian_tokenizer)

    output_data.append({
        'Original English': original_english,
        'Predicted Persian': predicted_persian,
        'Actual Persian': actual_persian
    })

    # Convert the predictions into a DataFrame and save to a CSV file
output_df = pd.DataFrame(output_data)
output_df.to_csv('translation_predictions_transformer.csv', index=False)

print("Predictions saved to 'translation_predictions_transformer.csv'")

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


BLEU Score: 0.02113649314394344
ROUGE-1: 0.0, ROUGE-L: 0.0
Predictions saved to 'translation_predictions_transformer.csv'
